Web Scratching Python Project - Vikram Singh

In [5]:
! pip install selenium

import requests
import os
import re
import time
import pandas as pd

In [7]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup

In [8]:
#creating folder

folder_name = 'games_html'
if not os.path.exists(folder_name):
    os.mkdir(folder_name)
#getting link of the game website
link = 'https://store.steampowered.com/games/'
response = requests.get(link)
response

<Response [200]>

In [12]:
#home page response content to a file in binary mode:

with open (os.path.join(folder_name, r'games_review.html'), mode='wb') as f:
    f.write(response.content)
os.listdir(folder_name)

['games_review.html']

In [19]:
#name of the game and corresponding URL's:

with open(os.path.join(folder_name,r'games_review.html'), encoding='UTF-8') as f:
    soup = BeautifulSoup(f, 'lxml')
    doc = soup.find('div',id='NewReleasesRows').find_all('a', limit=5, recursive=False)

game_list = []
for each in doc:
    name = each.find('div', class_='tab_item_name').contents[0]
    link = each['href']
    game_list.append({'title': name,
                    'link' : link
                   })
    
game_list

[{'title': 'Rick Henderson',
  'link': 'https://store.steampowered.com/app/1023790/Rick_Henderson/?snr=1_1452_4__103'},
 {'title': 'Power & Revolution 2021 Edition',
  'link': 'https://store.steampowered.com/app/1683320/Power__Revolution_2021_Edition/?snr=1_1452_4__103'},
 {'title': 'Here Comes Niko!',
  'link': 'https://store.steampowered.com/app/925950/Here_Comes_Niko/?snr=1_1452_4__103'},
 {'title': 'The Ramp',
  'link': 'https://store.steampowered.com/app/1506510/The_Ramp/?snr=1_1452_4__103'},
 {'title': "Hunter's Arena: Legends",
  'link': 'https://store.steampowered.com/app/1061100/Hunters_Arena_Legends/?snr=1_1452_4__103'}]

In [50]:
#game system requirements:

game_system_requirement = []
for each in game_list:
    response = requests.get(each['link'])
    #print(response.content)
    soup = BeautifulSoup(response.content, 'lxml')
    
    
    positive_review_count = ''
    try:
        positive_review_count = soup.find_all('div', class_='summary column')[0].find('span', class_='responsive_hidden').contents[0].strip()[1:-1]
    except:
        pass

    
    developer_name = ''
    try:
        developer_name = soup.find_all('div', class_='dev_row')[0].find('a').contents[0]
    except:
        pass
    
    
    publisher_name = ''
    try:
        publisher_name = soup.find_all('div', class_='dev_row')[1].find('a').contents[0]
    except:
        pass
    
    
    system_requirement={}
    count = 1
    for li in soup.find('div', {"class":['game_area_sys_req_full', 'game_area_sys_req_leftCol']}).find('ul', class_='bb_ul').find_all('li'):
        try:
            key, value = li.text.split(':')
        except:
            key = 'other_system_requirements'+str(count)
            value = li.text
        system_requirement[key] = value

        
        
    summary = {
        'title' : each['title'],
        'link': each['link'],
        'positive_review_count': review_count,
        'developer_name': developer_name,
        'publisher_name': publisher_name,
        }

    appended = {**summary, **system_requirement}

    game_system_requirement.append(appended)
    
    
game = pd.DataFrame(game_system_requirement) 
game

,title,link,positive_review_count,developer_name,publisher_name,OS,Processor,Memory,Graphics,DirectX,Storage,Sound Card,other_system_requirements1,Network
0,Rick Henderson,https://store.steampowered.com/app/1023790/Ric...,44,Fat Pug Studio,Fat Pug Studio,Microsoft® Windows® XP / Vista / 7 / 8 / 10,2.4 Ghz,8 GB RAM,Nvidia GT710/AMD RX550,Version 11,880 MB available space,Not required,NaN,NaN
1,Power & Revolution 2021 Edition,https://store.steampowered.com/app/1683320/Pow...,44,Eversim,Eversim,"Windows 10, 8, 7",1.6 GHZ,4 GB RAM,NaN,NaN,6 GB available space,NaN,Requires a 64-bit processor and operating system,NaN
2,Here Comes Niko!,https://store.steampowered.com/app/925950/Here...,44,Frog Vibes,Gears for Breakfast,Windows 10,3.0 GHz processor,4 GB RAM,NaN,NaN,2 GB available space,NaN,Requires a 64-bit processor and operating system,NaN
3,The Ramp,https://store.steampowered.com/app/1506510/The...,44,Hyperparadise,Hyperparadise,Windows 10,2.7 GHz Duo Core,4 GB RAM,2 GB VRAM,NaN,900 MB available space,You don't really need one. Just humming your ...,Requires a 64-bit processor and operating system,NaN
4,Hunter's Arena: Legends,https://store.steampowered.com/app/1061100/Hun...,44,Mantisco,Mantisco,"64-bit Windows 7, Windows 8.1, Windows 10",Intel Core i5-4430 / AMD FX-6300,8 GB RAM,NVIDIA GeForce GTX 770 / AMD Radeon R7 370 2GB,Version 11,30 GB available space,NaN,NaN,Broadband Internet connection


In [33]:
chrome_options = Options()
chrome_options.add_argument("--headless")

In [35]:
#reviews:

for each in game_system_requirement:
    #instantiating a chrome driver
    driver = webdriver.Chrome("C:\\Users\\OM PRAKASH PRAJAPATI\\Downloads\\chromedriver_win32\\chromedriver.exe", options=chrome_options)
    
    #getting the contents of the link
    driver.get(each['link'])

    #finding the element with class_name = 'review_box'
    element = driver.find_element_by_class_name("review_box")

    #instantiating chrome action
    action = ActionChains(driver)

    #scrolling down to execute the jquery script and load reviews
    action.move_to_element(element).perform()

    #waiting for 3 seconds for reviews to load
    time.sleep(3)

    #getting all review values returned by jquery response
    reviews = driver.find_elements_by_css_selector(".review_box .content")

    all_reviews = ''

    for each_review in reviews:
        try:
            all_reviews = all_reviews + '\n' + each_review.text
        except:
            pass

    #quitting the driver
    driver.quit()
    
    #adding reviews to each game dict object
    each['reviews'] = all_reviews

In [48]:
#exporting to csv

game.to_csv(os.path.join(folder_name, r'games_project.csv'), index=False)